# Tutorial 09 - Grouping and Aggregating - Part 1

The real power of data analysis with `DataFrames` comes into focus when we start utilizing the `.groupby()` and `.agg()` methods.  

This is known as *grouping* and *aggregating*.

Talking about grouping in the abstract can be confusing; I think it's best to see grouping in action by doing meaningful calculations.

The purpose of this tutorial is to introduce grouping and aggregation by way of following finance task: calculating monthly returns and volatilities for several assets.

## Loading Packages

Let's load the packages that we will need for this tutorial.

In [1]:
import numpy as np
import pandas as pd

## Reading-In Data

The analysis we will be performing will be on the set of of December 2018 prices for `SPY`, `IWM`, `QQQ`, `DIA`.

Let's read that data in from the CSV file.

In [2]:
df_etf = pd.read_csv("../data/index_etf_dec_2018.csv")
df_etf.head()

,symbol,date,open,high,low,close,volume,adjusted
0,SPY,2018-11-30,273.809998,276.279999,273.450012,275.649994,98204200,271.527222
1,SPY,2018-12-03,280.279999,280.399994,277.510010,279.299988,103176300,275.122589
2,SPY,2018-12-04,278.369995,278.850006,269.899994,270.250000,177986000,266.207977
3,SPY,2018-12-06,265.920013,269.970001,262.440002,269.839996,204185400,265.804108
4,SPY,2018-12-07,269.459991,271.220001,262.630005,263.570007,161018900,259.627899


**Coding Challenge:** Use `DataFrame` masking to isolate all the `QQQ` rows.

## Calculating Daily Log-Returns with `groupby()`

Our ultimate goal is to calculate monthly returns and monthly volatilities for each ETF in `df_etf`.  These quantities are both functions of daily returns.  So, our first order of business is to calculate daily returns. 

In a previous tutorial we calculated daily returns in a vectorized fashion.  Unfortunately, we can't use the exact same approach here because there are multiple ETFs in the data set.

To overcome this challenge we will use our first application of `groupby()`.

Here is the vectorized code that calculated daily log-returns for each ETF.

In [3]:
# sorting values to get everything in the right order
df_etf.sort_values(['symbol', 'date'], inplace=True)

# vectorized return calculation
df_etf['ret'] = \
    df_etf['close'].groupby(df_etf['symbol']).pct_change()
df_etf.head()

,symbol,date,open,high,low,close,volume,adjusted,ret
60,DIA,2018-11-30,253.100006,255.839996,252.830002,255.509995,2991800,251.157043,NaN
61,DIA,2018-12-03,260.170013,260.299988,257.019989,258.730011,9544200,254.322189,0.012602
62,DIA,2018-12-04,257.640015,258.040009,250.380005,250.729996,5376900,246.458496,-0.030920
63,DIA,2018-12-06,246.360001,249.869995,242.820007,249.720001,8232800,245.465698,-0.004028
64,DIA,2018-12-07,249.660004,251.410004,243.289993,244.309998,5963700,240.147842,-0.021664


**Code Challenge:** If the `group_by()` worked correctly, we should see value of `NaN` in the `log_ret` column for the first trade-date of each ETF.  Use `DataFrame` masking to confirm this.

The `group_by()` function actually saves us from a fair amount of hassle, as we see next.

The following code calculates the daily return for each day and each symbol using a `for` loop.  It is similar to the code we used in a previous tutorial except there is some logic to deal with the fact that there are different symbols in the same data set.  For the purposes of this tutorial you can simply run the code.  If you are curious about it, I encourage you to analyze it on your own.

In [4]:
df_etf['ret_loop'] = np.nan
for ix in range(1, df_etf.shape[0]):
    
    # grabbing symbols from df_etf
    curr_sym = df_etf.at[ix, 'symbol']
    prev_sym = df_etf.at[ix-1, 'symbol']
    
    # grabbling prices from df_etf
    curr_px = df_etf.at[ix, 'close']
    prev_px = df_etf.at[ix-1, 'close']
    
    # calculating return
    if curr_sym == prev_sym:
        df_etf.at[ix, 'ret_loop'] = (curr_px / prev_px) - 1

df_etf.head()

,symbol,date,open,high,low,close,volume,adjusted,ret,ret_loop
60,DIA,2018-11-30,253.100006,255.839996,252.830002,255.509995,2991800,251.157043,NaN,NaN
61,DIA,2018-12-03,260.170013,260.299988,257.019989,258.730011,9544200,254.322189,0.012602,0.012602
62,DIA,2018-12-04,257.640015,258.040009,250.380005,250.729996,5376900,246.458496,-0.030920,-0.030920
63,DIA,2018-12-06,246.360001,249.869995,242.820007,249.720001,8232800,245.465698,-0.004028,-0.004028
64,DIA,2018-12-07,249.660004,251.410004,243.289993,244.309998,5963700,240.147842,-0.021664,-0.021664


As you can see, both return methodologies yield the same result.  The vectorized code is far more succinct (though this isn't always a good thing).

We'll now proceed to calculating monthly returns and volatility for each of the ETFs in our data set.  This amounts to first grouping by `symbol`, and then performing an aggregation calculation on `returns`.  

Let's start with monthly returns.

## Monthly Return for Each `symbol`

If you take a look at our data set, we now have daily returns for each day, for each of the four ETFs.  Given a bit of time, you could probably come up with a `for` loop to iterate through the `DataFrame` and produce the monthly returns for each ETF (in fact, you could modify the returns `for` loop from the previous section to do just that).  As a one-off solution, that would be fine.  But grouping and aggregating are such ubiquitous operations, that it would be inconvenient to have write a `for`-loop every time.


Let's begin by calculating the daily growth factor in a separate column.

In [5]:
df_etf['growth_factor'] = 1 + df_etf['ret']
df_etf.head()

,symbol,date,open,high,low,close,volume,adjusted,ret,ret_loop,growth_factor
60,DIA,2018-11-30,253.100006,255.839996,252.830002,255.509995,2991800,251.157043,NaN,NaN,NaN
61,DIA,2018-12-03,260.170013,260.299988,257.019989,258.730011,9544200,254.322189,0.012602,0.012602,1.012602
62,DIA,2018-12-04,257.640015,258.040009,250.380005,250.729996,5376900,246.458496,-0.030920,-0.030920,0.969080
63,DIA,2018-12-06,246.360001,249.869995,242.820007,249.720001,8232800,245.465698,-0.004028,-0.004028,0.995972
64,DIA,2018-12-07,249.660004,251.410004,243.289993,244.309998,5963700,240.147842,-0.021664,-0.021664,0.978336


Recall that the monthly growth factor is the product of the daily growth factors.  Here is a way to write all that logic in a single line using `.groupby()` and `.agg()`:

In [6]:
df_grouped_factor = \
    df_etf.groupby(['symbol'])['growth_factor'].agg([np.prod]).reset_index()
df_grouped_factor

,symbol,prod
0,DIA,0.912684
1,IWM,0.877342
2,QQQ,0.910787
3,SPY,0.906657


Notice that `pandas` isn't very sophisticated about the name that it gives to the column that stores the aggregation calculation column.  It just gave it the name `prod`, which the name of the function that was used in the aggregation calculation.  Let's make `df_grouped_factor` a bit more readable by renaming that column.

In [7]:
df_grouped_factor.rename(columns={'prod': 'monthly_factor'}, inplace=True)
df_grouped_factor

,symbol,monthly_factor
0,DIA,0.912684
1,IWM,0.877342
2,QQQ,0.910787
3,SPY,0.906657


And finally, recall that the monthly return is calculated by subtracting one from the monthly growth factor.

In [8]:
df_grouped_factor['monthly_return'] = df_grouped_factor['monthly_factor'] - 1
df_grouped_factor[['symbol', 'monthly_return']]

,symbol,monthly_return
0,DIA,-0.087316
1,IWM,-0.122658
2,QQQ,-0.089213
3,SPY,-0.093343


## Monthly Volatility for Each `symbol`

Now let's calculate the (realized/historical) volatility for each of the ETFs.  Recall that volatility is the standard deviation of the daily returns.  If we were to do this in a brute force manner, we could use `DataFrame` masking to separate out the returns for each of the four ETFs, and then calculate four separate standard deviations.

However, once again with the power of `.groupby()` and `.agg()` we can do all of this with a single line of code.

In [9]:
df_grouped_vol = \
    df_etf.groupby(['symbol'])['ret'].agg([np.std]).reset_index()

df_grouped_vol

,symbol,std
0,DIA,0.019042
1,IWM,0.019526
2,QQQ,0.024004
3,SPY,0.019173


Again, let's rename our aggregation column to something more descriptive.

In [10]:
df_grouped_vol.rename(columns={'std':'daily_vol'}, inplace=True)

What we have calculated is a daily volatility, but when practitioners talk about volatility, the typically annualize it.  A daily volatility is annualized by multiplying by $\sqrt{252}$.

In [11]:
df_grouped_vol['ann_vol'] = df_grouped_vol['daily_vol'] * np.sqrt(252)
df_grouped_vol

,symbol,daily_vol,ann_vol
0,DIA,0.019042,0.302287
1,IWM,0.019526,0.309960
2,QQQ,0.024004,0.381056
3,SPY,0.019173,0.304359


**Code Challenge** Use `groupby()` and `.agg()` to calculate the average daily return for each of the ETFs.

## Related Reading

*PDSH* - 3.8 - Aggregation and Grouping

*Python for Data Analysis (McKinney)* - Chapter 9 (pp 251-274) Data Aggregation and Grouping Operations

*Options, Futures, and Other Derivatives (Hull)* - Chapter 15 (pp 325-329) The Black-Scholes-Merton Model